This inference uses 2/10 generation and pyMC

In [1]:
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns 
from scipy.stats import beta, bernoulli
import numpy as np
import pymc as pm

rho = 0.8
pop_size = 5000
epsilon = 0.05
pi = 0.5

def generate(rho=rho,  N=pop_size, epsilon=epsilon, pi=pi):
    sigma = 0.175 * (rho ** 2) - 0.3625 * rho + 0.1875
    a = rho * ((rho * (1 - rho)) / sigma - 1)
    b = (1 - rho) * ((rho * (1 - rho)) / sigma - 1)

    weights = [pi, 1-pi]
    mixture_samples = np.random.choice([0, 1], size=N, p=weights)
    u = 2 * np.where(mixture_samples == 0, beta.rvs(a, b, size=N), beta.rvs(b, a, size=N)) - 1
    p_y = (1 + u) / 2  
    y = bernoulli.rvs(p_y)  
    phi = np.vstack([(1 - (u+1)/2) * (1 - epsilon), (u+1)/2 * (1 - epsilon), np.repeat(epsilon, N)]).T
    
    # Known and unknown variables
    known = (y, phi)
    unknown = (a, b, epsilon, u)
    
    return known, unknown

known, unknown = generate()

a, b, e, u = unknown
y, phi = known
print(f"True Alpha: {a}")
print(f"True Beta: {b}")
print(f"True Epsilon: {e}")

def run_inference(y, phi, method='MCMC', steps=1000, tune=500, target_accept=0.95, mu_W=0, sigma_W=3, shape_W=(3, 1)):
    with pm.Model() as model:
        # Define priors for 'a' and 'b' as random variables to be inferred
        a = pm.HalfNormal('a', sigma=10)  # HalfNormal is often used as a prior for scale parameters
        b = pm.HalfNormal('b', sigma=10)

        # Now 'a' and 'b' are part of the model and will be inferred from the data
        u_raw = pm.Mixture('u_raw', w=[0.5, 0.5],
                           comp_dists=[pm.Beta.dist(a, b), pm.Beta.dist(b, a)])
        u = pm.Deterministic('u', 2 * u_raw - 1)


        # Adjusted: Model y based on the new Bernoulli relationship
        p_y = pm.Deterministic('p_y', (1 + u) / 2)
        y_likelihood = pm.Bernoulli('y_like', p=p_y, observed=y)

        # Prior for weights W with the option for prior guesses
        W = pm.Normal('W', mu=mu_W, sigma=sigma_W, shape=shape_W)

        # Calculate softmax probabilities for the first two categories
        softmax_probs = pm.math.softmax(pm.math.dot(u, W), axis=1)
        
        # Use softmax_probs directly in phi_likelihood
        phi_likelihood = pm.Categorical('phi_like', p=softmax_probs, observed=phi)

        if method == 'MCMC':
            trace = pm.sample(steps, tune=tune, target_accept=target_accept, progressbar=True)
            
            summary = az.summary(trace)
            print(summary)
            az.plot_trace(trace)
            plt.show()

            ppc = pm.sample_posterior_predictive(trace, model=model)
            az.plot_ppc(az.from_pymc3(posterior_predictive=ppc, model=model))
            plt.show()
        elif method == 'VI':
            approx = pm.fit(n=steps, method='fullrank_advi', progressbar=True)
            trace = approx.sample(draws=steps)
            summary = az.summary(trace)
            print(summary)
            az.plot_trace(trace)
            plt.show()

            ppc = pm.sample_posterior_predictive(trace, model=model)
            print(ppc)

    return trace


True Alpha: 12.673684210526261
True Beta: 3.1684210526315644
True Epsilon: 0.05


In [3]:
trace = run_inference(y, phi, method='MCMC')

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, b, u_raw, W]


Sampling 4 chains for 500 tune and 1_000 draw iterations (2_000 + 4_000 draws total) took 29 seconds.


NameError: name 'az' is not defined